# Notebook for converting .mib files to .hspy with the mib2hspy tools
This notebook explains the basic working principle behind mib2hspy and how the various parts interact with eachother.

In [1]:
%matplotlib qt
%config Completer.use_jedi = False #Autocomplete is sometimes slow, this disables some stuff to make it faster

import mib2hspy as m2h
import pandas as pd
import hyperspy.api as hs
from math import nan

## Prepare metadata
The first step is to define metadata and load the data. It is advised you do this in the same code block, as it ensures that the metadata you are currently working with is the one you want for your data that is presently loaded. However, in this example, we first create a blank `Converter` object and then set the metadata in it. The data will be loaded at a later stage

In [2]:
converter = m2h.Converter()
print(converter)

Converter for file "None" with data None:

***Header content***
None***

***Microscope parameters***
Parameter             Value    Units    Nominal value
--------------------  -------  -------  ---------------
Acceleration Voltage  nan      V
Mode                  None
Alpha                 nan
Magnification         nan               nan
Scale                 nan      nm
Cameralength          nan      cm       nan
Scale                 nan      1/Å
Mag mode              None
Rocking angle         nan      deg      nan
Rocking frequency     nan      Hz
Step Y                nan      nm       nan
Step X                nan      nm       nan
Convergence angle     nan      mrad     nan
Condenser aperture    nan      um       nan
Spot                  nan
Spotsize              nan      nm       nan
Acquisition Date      None
Camera                None
Exposure time         nan      ms
Microscope            None


As you can see, the converter object is not currently set to work with a file or any data. The microscope parameters in the object is also either `None` or `nan`, indicating that they have not been set yet. The `Converter.microscope_parameters` object is actually a `MicroscopeParameters` object which is a little complicated, but allows for calibrating data in a relatively straightforward way. Each of the microscope parameters in a `MicroscopeParameters` object has a very specific type. They are all derived from the `m2h.Tools.parameters.Parameter` class. To set the various parameters, you can do:

In [3]:
converter.microscope_parameters.camera = 'Merlin'
converter.microscope_parameters.microscope = '2100F'
converter.microscope_parameters.mode = 'NBD'
converter.microscope_parameters.alpha = 1
converter.microscope_parameters.acceleration_voltage = 200000
converter.microscope_parameters.cameralength = (12, nan)
#converter.microscope_parameters.magnification = (10000, nan) #only cameralength or magnification should be set in order to avoid unwanted behaviour
#converter.microscope_parameters.mag_mode = 'MAG1' #Only set this if you are working in imaging mode.
converter.microscope_parameters.scan_step_x = (16, 16)
converter.microscope_parameters.scan_step_y = (16, 16)
converter.microscope_parameters.spotsize = (0.5, nan)
#converter.microscope_parameters.spot = 1 #Only use spotsize or spot, depends on the mode you are working on
#converter.microscope_parameters.rocking_angle = (1, nan)
#converter.microscope_parameters.rocking_frequency = 100
print(converter.microscope_parameters)

Parameter             Value    Units    Nominal value
--------------------  -------  -------  ---------------
Acceleration Voltage  200000   V
Mode                  NBD
Alpha                 1
Magnification         nan               nan
Scale                 nan      nm
Cameralength          nan      cm       12
Scale                 nan      1/Å
Mag mode              None
Rocking angle         nan      deg      nan
Rocking frequency     nan      Hz
Step Y                16       nm       16
Step X                16       nm       16
Convergence angle     nan      mrad     nan
Condenser aperture    nan      um       nan
Spot                  nan
Spotsize              nan      nm       0.5
Acquisition Date      None
Camera                Merlin
Exposure time         nan      ms
Microscope            2100F


To set the values of the objects based on their nominal values and a calibration table, you can do:

In [4]:
table = pd.read_excel(r'Calibrations.xlsx', engine='openpyxl') #The calibration table file must be in a very specific format to be useable by the parameters object.
converter.microscope_parameters.set_values_from_calibrationtable(table)
print(converter.microscope_parameters)

Parameter             Value                 Units    Nominal value
--------------------  --------------------  -------  ---------------
Acceleration Voltage  200000                V
Mode                  NBD
Alpha                 1
Magnification         nan                            nan
Scale                 nan                   nm
Cameralength          22.95273094471488     cm       12
Scale                 0.009554593381028329  1/Å
Mag mode              None
Rocking angle         nan                   deg      nan
Rocking frequency     nan                   Hz
Step Y                16                    nm       16
Step X                16                    nm       16
Convergence angle     nan                   mrad     nan
Condenser aperture    nan                   um       nan
Spot                  nan
Spotsize              nan                   nm       0.5
Acquisition Date      None
Camera                Merlin
Exposure time         nan                   ms
Microscope       

This raises alot of warning if the values or names are not present in the calibration table, but we see that it has found a calibration for the cameralength and defined the diffraction scale! For the actual conversion, only the Diffraction scale or the image scale and the scan step sizes are important (they will be used to set appropriate details in the `axes_manager`), but it is advised to set as much as possible to save you work in the long run.

## Load data
Now the time has come to load the data. We first assign the converter a path to the data we want to convert, then read the data. Only valid .mib data paths are accepted by the converter to avoid unpleasent surprises later on. The data is also read lazily, so it should take much time.

To ensure that the data path is always the correct path, only valid .mib data paths can be used as the data path in a `Converter` object. Furthermore, whenever you change the `Converter.data_path` property, the `Converter.data` will be deleted. This way, you can be fairly certain that the `data_path` in your `Converter` object always correspond to the currently loaded data!

In [5]:
data_path = r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Merlin\LFO\2021_03_18_P40204PV04\SED_128x128x12_16x16nm_NBD_alpha1_spot05nm_CL3-7D11_IL1-4C92_CL12cm.mib'
converter.data_path = data_path
converter.read_mib()
print(converter)

This mib file appears to be TEM data. The stack is returned with no reshaping.
Converter for file "C:\Users\emilc\OneDrive - NTNU\NORTEM\Merlin\LFO\2021_03_18_P40204PV04\SED_128x128x12_16x16nm_NBD_alpha1_spot05nm_CL3-7D11_IL1-4C92_CL12cm.mib" with data <LazyElectronDiffraction2D, title: SED_128x128x12_16x16nm_NBD_alpha1_spot05nm_CL3-7D11_IL1-4C92_CL12cm, dimensions: (16384|256, 256)>:

***Header content***
Content of Medipix HDR file "C:\Users\emilc\OneDrive - NTNU\NORTEM\Merlin\LFO\2021_03_18_P40204PV04\SED_128x128x12_16x16nm_NBD_alpha1_spot05nm_CL3-7D11_IL1-4C92_CL12cm.hdr":
	Time and Date Stamp (day, mnth, yr, hr, min, s): 18/03/2021 19:30:37
	Chip ID: W559_G11, - , - , -
	Chip Type (Medipix 3.0, Medipix 3.1, Medipix 3RX): Medipix 3RX
	Assembly Size (NX1, 2X2): 1x1
	Chip Mode  (SPM, CSM, CM, CSCM): SPM
	Counter Depth (number): 12
	Gain: SLGM
	Active Counters: Counter 0
	Thresholds (keV): 4.000000E+1,5.110000E+2,0.000000E+0,0.000000E+0,0.000000E+0,0.000000E+0,0.000000E+0,0.000000E+0


We see that the converter has loaded the data as well as the available header file. Let us inspect the converter a little bit closer:

In [6]:
print(converter.frames)
print(converter.dimension)
print(converter.ndx)
print(converter.ndy)
print(converter.nx)
print(converter.ny)

16384
3
256
256
16384
1


We see that the converter has 65536 frames, dimension 3, and the detector sizes are 256x256. We also see that the number of scan pixels in the x-direction matches the number of frames and that the number of y-pixels are 1. That sounds about right. When loading mib data, it is loaded as a 3D stack, and we need to reshape it ourselves.
### Reshaping the data

In [7]:
converter.reshape(128, 128)
print(converter.frames)
print(converter.dimension)
print(converter.ndx)
print(converter.ndy)
print(converter.nx)
print(converter.ny)

16384
4
256
256
128
128


Now the number of scan pixels look more sensible. The converter object would throw an error if you tried to reshape the data into an inappropriate stack, so it ensures that `nx*ny=frames`. Let us now take a closer look at how the data in the converter object is chunked.
### Chunking
To see the data array in the signal of the converter, do:

In [8]:
converter.data.data

dask.array<reshape, shape=(128, 128, 256, 256), dtype=>u2, chunksize=(5, 128, 256, 256), chunktype=numpy.ndarray>

Here we see that the chunking is very wierd and off. We should therefore rechunk our data so it makes more sense and is more effective when loading the final converted data lazily:

In [9]:
converter.rechunk(32)
converter.data.data

,Array,Chunk
Bytes,2.00 GiB,2.00 MiB
Shape,"(128, 128, 256, 256)","(32, 32, 32, 32)"
Count,3007 Tasks,1024 Chunks
Type,>u2,numpy.ndarray


Now the dask array looks more sensible. Each chunk has the same shape and measure 32x32x32x32. The final step in preparing your data is to apply the calibrations and set the metadata of the signal.
### Applying calibrations
You must specifically ask the calibrations in the `microscope_parameter` to apply to your data. Let us take a closer look:

In [10]:
print(converter.data.axes_manager)
print(converter.data.metadata)
print(converter.data.original_metadata)

<Axes manager, axes: (128, 128|256, 256)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
     <undefined> |    128 |      0 |       0 |       1 | <undefined> 
     <undefined> |    128 |      0 |       0 |       1 | <undefined> 
---------------- | ------ | ------ | ------- | ------- | ------ 
     <undefined> |    256 |        |       0 |       1 | <undefined> 
     <undefined> |    256 |        |       0 |       1 | <undefined> 
├── General
│   └── title = 
└── Signal
    ├── binned = False
    └── signal_type = electron_diffraction




We see that no calibration data has been set yet and there are no metadata present, so let us apply the calibrations

In [11]:
converter.apply_calibrations()
print(converter.data.axes_manager)
print(converter.data.metadata)
print(converter.data.original_metadata)

<Axes manager, axes: (128, 128|256, 256)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    128 |      0 |       0 |      16 |     nm 
               y |    128 |      0 |       0 |      16 |     nm 
---------------- | ------ | ------ | ------- | ------- | ------ 
              kx |    256 |        |    -1.2 |  0.0096 | $A^{-1}$ 
              ky |    256 |        |    -1.2 |  0.0096 | $A^{-1}$ 
├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── Diffraction
│       │       └── camera_length = 22.95273094471488
│       ├── beam_energy = 200000
│       ├── convergence_angle = nan
│       ├── rocking_angle = nan
│       └── rocking_frequency = nan
├── General
│   └── title = 
└── Signal
    ├── binned = False
    └── signal_type = electron_diffraction

├── Acquisition_instrument
│   └── Parameters
│       ├── acceleration_voltage
│       │   ├── Units = V
│       

Now the axes have been calibrated, and there are relevant metadata information in both the converter.data.metadata field and in the converter.data.original_metadata field. The data should now be ready for writing.

### Writing data
To write the data into a .hspy file, simply do

In [12]:
converter.write('.hspy', overwrite=True)

You can also write the data into a .hdf5 file (identical to .hspy really), or to images such as .png or .tif. The image formats are most useful for single-frame data, but you can also ask for a set number of frames to be written:

In [47]:
converter.write('.png', num_frames=3)

C:\Users\emilc\Documents\mib2hspy\mib2hspy\Tools\conversion.py:751: UserWarning: <LazyElectronDiffraction2D, title: , dimensions: (256, 256|256, 256)> contains a stack of images. Converting this data to .png is not advised as it will create 65536 files
  warn(
C:\Users\emilc\Documents\mib2hspy\mib2hspy\Tools\conversion.py:498: UserWarning: Preparing plots for a stack is not advised.
  warn('Preparing plots for a stack is not advised.')


You can also ask for a VBF of your data and save it, but it is quite slow at the moment.

In [ ]:
fig = converter.plot_vbf(vbf_kwargs={'width':20})
fig.savefig(converter.data_path.with_suffix('.png'))

To load the converted data to check that everything went ok, you can load it like this. To inspect the data, it is relatively fast to create a rough square-VBF of the lazy data and look at that

In [48]:
s = hs.load(converter.data_path.with_suffix('.hspy'), lazy=True)
navigator = s.isig[-0.1:0.1, -0.1:0.1].sum(axis=[2, 3]).T
navigator.compute()
navigator.plot(cmap='inferno')

[########################################] | 100% Completed |  1.8s


C:\Users\emilc\Miniconda3\envs\mib2hspy\lib\site-packages\hyperspy\drawing\utils.py:163: MatplotlibDeprecationWarning: 
The set_window_title function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use manager.set_window_title or GUI-specific methods instead.
  fig.canvas.set_window_title(window_title)


## MicroscopeParameters Example
This part is meant to explain the workings of the microscope parameters objects that are used to calibrate the data. They are designed to work with a GUI, so they might appear overly complicated at first glance. It is best to use an example to explain how they work.

In the case of a scan dataset, the scan step along the x-direction should be stored in `converter.microscope_parameters.scan_step_x`. Taking a closer look at this object, we see that it has a name, a value, a nominal value, and a unit property:

In [15]:
print(converter.microscope_parameters.scan_step_x)
print(converter.microscope_parameters.scan_step_x.name)
print(converter.microscope_parameters.scan_step_x.value)
print(converter.microscope_parameters.scan_step_x.nominal_value)
print(converter.microscope_parameters.scan_step_x.units)

ScanStep Step X: nan (nan) nm
Step X
nan
nan
nm


Other parameters may or may not have the `.nominal_value` property, but the other properties are common for all the other microscope parameters.

Now, to set the value and the nominal value of the scan step, we can simply do:

In [18]:
converter.microscope_parameters.scan_step_x = (2, 1.99)
print(converter.microscope_parameters.scan_step_x)

ScanStep Step X: 1.99 (2) nm


If we only want to set the value, we can instead do:

In [20]:
converter.microscope_parameters.scan_step_x = 1.98
print(converter.microscope_parameters.scan_step_x)

ScanStep Step X: 1.98 (2) nm


As you can see, the value can be set by simple assignment, but the nominal value has to be set along with the actual value. That is because if you change the nominal value, you typically also want to change the actual value as well. To set only the nominal value, without specifying an actual value, simply set the actual value to `nan`:

In [21]:
converter.microscope_parameters.scan_step_x = (1.5, nan)
print(converter.microscope_parameters.scan_step_x)

ScanStep Step X: nan (1.5) nm


Some of the parameters, like `converter.microscope_parameters.scan_step_x` have additional properties that depend on the parameter. For example, the direction of the scan is defined in the `.direction` property (which exists only for the scan step parameters):

In [22]:
converter.microscope_parameters.scan_step_x.direction

'X'

In other words, each parameter object is made to work in a specific way and interact with the data you want to convert.


# Calibrations [WIP]

In [ ]:
signal = hs.load(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Calibrations\2100F\Merlin\2020_11_21_TEDPELLA673\CL8cm.hspy')
cl = m2h.get_calibration_from_MERLIN(signal)
print(cl)
cl.as_dataframe()

In [ ]:
signal = hs.load(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Calibrations\2100F\Merlin\2020_11_21_TEDPELLA673\SAMAG10k.hspy')
mag = m2h.get_calibration_from_MERLIN(signal)
mag.nominal_value=10000
mag.parameters['Mag mode']='SAMAG'
print(mag)
mag.as_dataframe()

In [ ]:
signal.original_metadata

In [ ]:
cameralengths = [
    (8, 16.2)
]

cl1 = m2h.Cameralength(8, 16.2, '2020-12-07',acceleration_voltage=200000, camera='Merlin')
#cl1.calibrate_cameralength()
cl1.calibrate_scale()
print(cl1.as_dataframe())

In [ ]:
mag1 = m2h.Magnification(12000, 1, '2020-12-07', scale=0.979, acceleration_voltage=200000, camera='US1000')
#mag1.calibrate_scale()
mag1.calibrate_magnification()
print(mag1)

In [ ]:
m1 = m2h.Magnification(8000, 12030, '2020-12-07', acceleration_voltage=200000, mode='TEM', camera='Ultrascan', mag_mode='SAMAG')
cl1 = m2h.Cameralength(8, 16.2, '2020-12-07', scale=0.00134, spot_size=0.5, acceleration_voltage=200000, camera='Merlin')
cl2 = m2h.Cameralength(10, 19.3, '2020-12-07', acceleration_voltage=200000, camera='Merlin')
cl3 = m2h.Cameralength(10, 19.3, '2020-12-07', acceleration_voltage=200000, camera='Ultrascan')
step1 = m2h.StepSize(2.5, 2.4, '2020-12-07', direction='X', mode='STEM', alpha=None)
precession_calibration = m2h.PrecessionAngle(1.0, 1.04, 20., '2020-12-07', mode='NBD', alpha='Alpha 4', deflectors={'Upper_1': {'X': {'A':0.1, 'P':39}, 'Y': {'A': 0.5, 'P': 44}}})
spot1 = m2h.Spotsize(3, '2020-12-07', spot_size=0.5, mode='NBD', alpha=4, acceleration_voltage=200000)
calibrations = m2h.CalibrationList()
calibrations+=m1
calibrations+=cl1
calibrations+=cl2
calibrations+=cl3
calibrations+=step1
calibrations += precession_calibration
calibrations += spot1
#print([calibration.nominal_value for calibration in calibrations])
print(calibrations.dataframe)


In [ ]:
cl1.as_dataframe()
#spot1.as_dataframe()

In [ ]:
new_calibrations = m2h.generate_from_dataframe(calibrations.dataframe)
print(new_calibrations.dataframe)

In [ ]:
precession_calibration.as_dataframe(ignore_nans=False)

In [ ]:
step1 = m2h.StepSize(2, 1.9, '2020-12-07', mode='STEM', alpha=None)
print(step1.as_dataframe())

In [ ]:
cl3.scale

In [ ]:
print(calibrations['`Nominal Cameralength (cm)`==8.0'])

In [ ]:

#print(precession_calibration)
print(precession_calibration.as_dataframe(ignore_nans=False))

In [ ]:
from mib2hspy import DiffractionScale

In [ ]:
cl3.scale

In [ ]:
cl3.scale

In [ ]:
print(calibrations['`Nominal Cameralength (cm)`==8.0'])

In [ ]:

#print(precession_calibration)
print(precession_calibration.as_dataframe(ignore_nans=False))

In [ ]:
from mib2hspy import DiffractionScale

In [ ]:
cl3.scale

In [ ]:
print(calibrations['`Nominal Cameralength (cm)`==8.0'])

In [ ]:

#print(precession_calibration)
print(precession_calibration.as_dataframe(ignore_nans=False))

In [ ]:
from mib2hspy import DiffractionScale

In [ ]:
cl = DiffractionScale(0.000135)
print(cl)
print(cl.to_inv_nm(200000))
print(cl.calculate_cameralength(200000, 55E-6))
dp = m2h.Cameralength(8, 16.20, '2020-11-20', scale=cl, acceleration_voltage=200000)

In [ ]:
print(dp.as_dataframe())

In [ ]:
mag1 = m2h.Magnification(8000, 16302, '2020-12-01', scale=m2h.Scale(0.34, 'nm'))
mag2 = m2h.Magnification(8000, 16302, '2020-12-01')
cl1 = m2h.Cameralength(8, 16.32, '2020-12-02', scale=m2h.Scale(0.34, '1/nm'), Acceleration_voltage=200000)

df = pd.DataFrame()
df = mag1.add_to_dataframe(df)
df = mag2.add_to_dataframe(df)
df = cl1.add_to_dataframe(df)

print(df)

In [ ]:
cl = DiffractionScale(0.000135)
print(cl)
print(cl.to_inv_nm(200000))
print(cl.calculate_cameralength(200000, 55E-6))
dp = m2h.Cameralength(8, 16.20, '2020-11-20', scale=cl, acceleration_voltage=200000)

In [ ]:
print(dp.as_dataframe())

In [ ]:
mag1 = m2h.Magnification(8000, 16302, '2020-12-01', scale=m2h.Scale(0.34, 'nm'))
mag2 = m2h.Magnification(8000, 16302, '2020-12-01')
cl1 = m2h.Cameralength(8, 16.32, '2020-12-02', scale=m2h.Scale(0.34, '1/nm'), Acceleration_voltage=200000)

df = pd.DataFrame()
df = mag1.add_to_dataframe(df)
df = mag2.add_to_dataframe(df)
df = cl1.add_to_dataframe(df)

print(df)

In [ ]:
df = mag1.as_dataframe()
df = mag2.add_to_dataframe(df)
print(df)


In [ ]:
df = mag1.as_dataframe()
df = mag2.add_to_dataframe(df, remove_duplicates=True)
print(df)
#print(mag)

In [ ]:
microscope = m2h.Microscope()

print(microscope)

In [ ]:
microscope.set_acceleration_voltage(200)
microscope.set_alpha(4)
microscope.set_mode('NBD')
microscope.set_nominal_cameralength(8)
microscope.set_cameralength(16.2)

print(microscope)
print(microscope.get_defined_parameters_(as_dict=True))